<a href="https://colab.research.google.com/github/Denev6/practice/blob/main/transformer/RoBERTa_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuned RoBERTa
- [EmoBERTa-large](https://github.com/tae898/erc)
- [MIT LICENSE](https://github.com/tae898/erc/blob/main/LICENSE)

In [ ]:
!python --version

import torch

print(f"PyTorch {torch.__version__}")

Python 3.8.16
PyTorch 1.13.0+cu116


In [ ]:
!pip install transformers

In [ ]:
import os
import gc
import warnings

import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, RobertaTokenizerFast, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount("/content/drive")
warnings.filterwarnings("ignore")

In [ ]:
def join_path(*args):
    return os.path.join("/content/drive/MyDrive", *args)


DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
TRAIN_CSV = join_path("data", "train.csv")
TEST_CSV = join_path("data", "test.csv")

ARGS = {
    "model": "tae898/emoberta-large",
    "model_path": join_path("model.pth"),
    "batch_size": 32,
    "grad_step": 8,
    "epochs": 10,
    "max_len": 128,
    "lr": 3e-7,
    "patience": 2,
}

## Utils

In [ ]:
class EarlyStopping(object):
    def __init__(self, patience, save_path):
        self._min_loss = np.inf
        self._patience = patience
        self._path = save_path
        self.__counter = 0

    def should_stop(self, model, loss):
        if loss < self._min_loss:
            self._min_loss = loss
            self.__counter = 0
            torch.save(model.state_dict(), self._path)
        elif loss > self._min_loss:
            self.__counter += 1
            if self.__counter >= self._patience:
                return True
        return False
   
    def load(self, model):
        model.load_state_dict(torch.load(self._path))
        return model
    
    @property
    def counter(self):
        return self.__counter

In [ ]:
class LabelEncoder(object):
    def __init__(self):
        self._targets = [
            "neutral",
            "joy",
            "surprise",
            "anger",
            "sadness",
            "disgust",
            "fear",
        ]
        self.num_classes = len(self._targets)

    def encode(self, label):
        return self._targets.index(label)

    def decode(self, label):
        return self._targets[label]

In [ ]:
class DataProcessor(object):
    def __init__(self, label_encoder):
        self._label_encoder = label_encoder

    def process(self, data, mode):
        data = data.reset_index(drop=True)
        if mode == "train":
            data["Target"] = data["Target"].map(self._label_encoder.encode)
            data = data.loc[:, ["Utterance", "Target"]]
        elif mode == "test":
            data = data.loc[:, ["ID", "Utterance"]]
        return data

In [ ]:
class RoBERTaDataset(Dataset):
    def __init__(
        self,
        data,
        roberta_tokenizer,
        max_length=512,
        mode=None,
    ):
        self._dataset = data
        self._roberta_tokenizer = roberta_tokenizer
        self._max_length = max_length
        self._mode = mode

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, idx):
        text = self._dataset.loc[idx, "Utterance"]
        inputs = self._roberta_tokenizer(
            text,
            max_length=self._max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        input_ids = inputs["input_ids"][0]
        attention_mask = inputs["attention_mask"][0]

        if self._mode == "train":
            y = self._dataset.loc[idx, "Target"]
            return input_ids, attention_mask, y
        else:
            return input_ids, attention_mask

## Dataset

In [ ]:
label_encoder = LabelEncoder()
processor = DataProcessor(label_encoder)
roberta_tokenizer = RobertaTokenizerFast.from_pretrained(ARGS["model"], truncation=True)

In [ ]:
train_csv = pd.read_csv(TRAIN_CSV)
df_train, df_val = train_test_split(
    train_csv, test_size=0.2, shuffle=True, random_state=32
)

In [ ]:
df_train = processor.process(df_train, mode="train")
df_val = processor.process(df_val, mode="train")

train_set = RoBERTaDataset(
    df_train,
    roberta_tokenizer,
    max_length=ARGS["max_len"],
    mode="train",
)
val_set = RoBERTaDataset(
    df_val,
    roberta_tokenizer,
    max_length=ARGS["max_len"],
    mode="train",
)

train_dataloader = DataLoader(train_set, batch_size=ARGS["batch_size"])
val_dataloader = DataLoader(val_set, batch_size=ARGS["batch_size"])

## Train

In [ ]:
def evaluate(model, criterion, val_loader, device, mode=None):
    model.eval()

    val_loss = list()
    model_preds = list()
    true_labels = list()

    with torch.no_grad():
        for input_ids, attention_mask, label in val_loader:
            label = label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output.logits, label.long())
            val_loss.append(batch_loss.item())

            model_preds += output.logits.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()


        if mode != "train":
            val_acc = accuracy_score(true_labels, model_preds)
            val_f1 = f1_score(true_labels, model_preds, average="macro")
            return val_acc, val_f1

        return val_loss


def train(model, optimizer, scheduler, criterion, train_loader, val_loader, device):
    torch.cuda.empty_cache()
    gc.collect()

    model_path = ARGS["model_path"]
    grad_step = ARGS["grad_step"]
    epoch_progress = trange(1, ARGS["epochs"] + 1)
    early_stopper = EarlyStopping(ARGS["patience"], model_path)

    model.to(device)
    criterion.to(device)
    model.zero_grad()

    for epoch in epoch_progress:

        model.train()
        train_loss = list()
        for batch_id, data in enumerate(train_loader, start=1):

            input_ids, attention_mask, train_label = data
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)
            train_label = train_label.to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output.logits, train_label.long())
            train_loss.append(batch_loss.item())

            batch_loss /= grad_step
            batch_loss.backward()

            if batch_id % grad_step == 0:
                optimizer.step()
                model.zero_grad()

        val_loss = evaluate(model, criterion, val_loader, device, mode="train")
        train_loss = np.mean(train_loss)
        val_loss = np.mean(val_loss)
        tqdm.write(
            f"Epoch {epoch}, Train-Loss: {train_loss:.5f},  Val-Loss: {val_loss:.5f}"
        )

        if early_stopper.should_stop(model, val_loss):
            break

        scheduler.step()

    tqdm.write(f"\n\n -- EarlyStopping: [Epoch: {epoch - early_stopper.counter}]")
    tqdm.write(f"Model saved at '{model_path}'.")
    model = early_stopper.load(model)

    return model

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(ARGS["model"])
optimizer = AdamW(
    model.parameters(), lr=ARGS["lr"], weight_decay=0.01, correct_bias=False
)
scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.9**epoch)
criterion = nn.CrossEntropyLoss()

In [ ]:
best_model = train(
    model, optimizer, scheduler, criterion, train_dataloader, val_dataloader, DEVICE
)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1, Train-Loss: 0.74695,  Val-Loss: 0.63894
Epoch 2, Train-Loss: 0.70502,  Val-Loss: 0.63519
Epoch 3, Train-Loss: 0.68978,  Val-Loss: 0.63589
Epoch 4, Train-Loss: 0.67846,  Val-Loss: 0.63311
Epoch 5, Train-Loss: 0.66376,  Val-Loss: 0.63379
Epoch 6, Train-Loss: 0.66584,  Val-Loss: 0.63418


 -- EarlyStopping: [Epoch: 4]
Model saved at '/content/drive/MyDrive/model.pth'.


In [ ]:
torch.cuda.empty_cache()
gc.collect()

val_acc, val_f1 = evaluate(best_model, criterion, val_dataloader, DEVICE)
print(f"Accuracy: {val_acc:.5f}")
print(f"F1-macro: {val_f1:.5f}")

Accuracy: 0.80280
F1-macro: 0.69021


## Prediction

In [ ]:
df_test = pd.read_csv(TEST_CSV)

df_test = processor.process(df_test, mode="test")
test_set = RoBERTaDataset(
    df_test,
    roberta_tokenizer,
    max_length=ARGS["max_len"],
)
test_dataloader = DataLoader(test_set, batch_size=ARGS["batch_size"], shuffle=False)

In [ ]:
def predict(model, test_loader, device):
    model.eval()
    model_preds = list()

    with torch.no_grad():
        for input_ids, attention_mask in test_loader:
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)
            model_preds += output.logits.argmax(1).detach().cpu().numpy().tolist()
        return model_preds

In [ ]:
preds = predict(best_model, test_dataloader, DEVICE)

df_test["Target"] = preds
df_test["Target"] = df_test["Target"].map(label_encoder.decode)
res = df_test.loc[:, ["Target"]]
res.head(10)

,Target
0,surprise
1,neutral
2,neutral
3,neutral
4,joy
5,joy
6,joy
7,joy
8,sadness
9,joy
